## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-22-17-00-00 +0000,wsj,Defining Moments in TV History You’ve Probably...,https://www.wsj.com/arts-culture/television/tv...
1,2026-01-22-16-59-23 +0000,nyt,Live Updates: Zelensky Criticizes European All...,https://www.nytimes.com/live/2026/01/22/us/tru...
2,2026-01-22-16-59-08 +0000,nyt,Betting on Prediction Markets Is Their Job. Th...,https://www.nytimes.com/2026/01/22/business/pr...
3,2026-01-22-16-54-59 +0000,nyt,Supreme Court Hearing Reveals Unease Over Thre...,https://www.nytimes.com/2026/01/21/us/politics...
4,2026-01-22-16-54-14 +0000,nypost,California carjacker mowed down and shot by co...,https://nypost.com/2026/01/22/us-news/san-jose...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2339/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,105
118,greenland,33
272,davos,33
68,ice,21
117,deal,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
331,2026-01-21-20-46-06 +0000,cbc,"Trump reiterates desire to acquire Greenland, ...",https://www.cbc.ca/news/world/trump-davos-spee...,216
203,2026-01-22-07-39-58 +0000,bbc,Trump says 'framework of a future deal' discus...,https://www.bbc.com/news/articles/cgezx40r7d7o...,208
370,2026-01-21-19-14-45 +0000,nyt,Did the U.S. Ever Own Greenland? Fact-Checking...,https://www.nytimes.com/2026/01/21/us/politics...,195
136,2026-01-22-11-19-40 +0000,nypost,"Trump claims ‘Iran does want to talk, and we’l...",https://nypost.com/2026/01/22/us-news/trump-cl...,191
112,2026-01-22-12-49-01 +0000,nyt,Zelensky Meets With Trump at Davos Forum to Di...,https://www.nytimes.com/2026/01/22/us/politics...,186


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
331,216,2026-01-21-20-46-06 +0000,cbc,"Trump reiterates desire to acquire Greenland, ...",https://www.cbc.ca/news/world/trump-davos-spee...
136,86,2026-01-22-11-19-40 +0000,nypost,"Trump claims ‘Iran does want to talk, and we’l...",https://nypost.com/2026/01/22/us-news/trump-cl...
240,70,2026-01-22-02-17-39 +0000,cbc,Former Uvalde school police officer found not ...,https://www.cbc.ca/news/world/uvalde-police-sc...
403,63,2026-01-21-17-25-34 +0000,nyt,Tariffs Case Pending as Supreme Court Takes Up...,https://www.nytimes.com/2026/01/21/us/politics...
60,59,2026-01-22-15-05-53 +0000,nypost,Two ringleaders of anti-ICE mob who stormed ch...,https://nypost.com/2026/01/22/us-news/alleged-...
231,56,2026-01-22-03-11-00 +0000,wsj,The Qatari-donated plane that is being overhau...,https://www.wsj.com/politics/national-security...
203,51,2026-01-22-07-39-58 +0000,bbc,Trump says 'framework of a future deal' discus...,https://www.bbc.com/news/articles/cgezx40r7d7o...
23,40,2026-01-22-16-26-35 +0000,startribune,7-year-old girl abducted in central Minnesota ...,https://www.startribune.com/7-year-old-girl-ab...
196,38,2026-01-22-08-43-21 +0000,wsj,"Stock Market Today: Dow Gains, Global Stocks R...",https://www.wsj.com/livecoverage/stock-market-...
330,38,2026-01-21-20-51-42 +0000,nypost,House committee finds Bill and Hillary Clinton...,https://nypost.com/2026/01/21/us-news/house-co...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
